In [14]:
from manim import *
from difflib import SequenceMatcher

In [43]:
def tokenize(code: Code) -> List[VMobjectFromSVGPath|Dot]:
    tokens = []
    for line in code[2]: # 0 = frame, 1 = dot, 2 = paragraph
        current_vmobject_group = []
        for vmobject in line:
            if isinstance(vmobject, Dot):
                if len(current_vmobject_group) > 0:
                    tokens.append(VGroup(*current_vmobject_group))
                    current_vmobject_group = []

                # tokens.append(vmobject)
                continue

            current_vmobject_group.append(vmobject)
            
        if len(current_vmobject_group) > 0:
            tokens.append(VGroup(*current_vmobject_group))

    return tokens

In [49]:
rust_code = """impl From<u64> for Health {
  fn from(bitfield: u64) -> Health {
    // ...
  }
}"""

token_replacements = """impl TryFrom<u64> for Health {
  type Error = anyhow::Error;
  fn from(bitfield: u64) -> Result<Self, Self::Error> {
    // ...
  }
}"""

In [51]:
sequence_matcher = SequenceMatcher(None, rust_code.split(), token_replacements.split())
opcodes = list(sequence_matcher.get_opcodes())
opcodes

[('equal', 0, 1, 0, 1),
 ('replace', 1, 2, 1, 2),
 ('equal', 2, 5, 2, 5),
 ('insert', 5, 5, 5, 9),
 ('equal', 5, 9, 9, 13),
 ('replace', 9, 10, 13, 15),
 ('equal', 10, 15, 15, 20)]

In [53]:
%%manim -qm -v WARNING CodeBlock

class CodeBlock(Scene):
    def construct(self):
        code = tokenize(Code(code=rust_code, language="rust", insert_line_no=False))
        replacement = tokenize(Code(code=token_replacements, language="rust", insert_line_no=False))

        writes = [Write(token) for token in code]
        self.play(AnimationGroup(writes, lag_ratio=0.30))
        self.wait()

        sequence_matcher = SequenceMatcher(None, rust_code.split(), token_replacements.split())
        opcodes = list(sequence_matcher.get_opcodes())

        unwrites = []
        for op, i0, i1, j0, j1 in opcodes:
            if op == 'delete' or op == 'replace':
                for x in range(i0, i1):
                    unwrites.append(Unwrite(code[x]))
        
        self.play(AnimationGroup(*unwrites))

        transforms = []
        for op, i0, i1, j0, j1 in opcodes:
            if op == 'equal':
                for x, y in zip(range(i0, i1), range(j0, j1)):
                    transforms.append(Transform(code[x], replacement[y]))

        self.play(AnimationGroup(*transforms))

        writes = []
        for op, i0, i1, j0, j1 in opcodes:
            if op == 'insert' or op == 'replace':
                for x in range(j0, j1):
                    writes.append(Write(replacement[x]))

        self.play(AnimationGroup(*writes, lag_ratio=0.30))
        self.wait()

Manim Community v0.18.0